Pairk - Pairwise k-mer alignment 

In [15]:
import pairk
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# load an example dataset

load an example alignment

In [16]:
ex1 = pairk.example1

This example is imported as an object but just contains: an MSA, the query id, positions of query idrs, and the IDR sequences from the MSA themselves.

In [17]:
print(ex1)

Alignment file: /Users/jackson/Dropbox (MIT)/work/07-SLiM_bioinformatics/11-pairk/pairk/data/example_alignment_9606_0_00294e-idraln-555_to_971-idr-440_to_665.fasta
Query ID: 9606_0:00294e
IDR start in MSA: 555
IDR end in MSA: 971IDR start: 440
IDR end: 665



For Pairk, you just need a set of IDR sequences in dictionary format.<br>

Here I defined the IDRs via the query sequence IDR positions, extracted the relevant sequences from the MSA, and then unaligned them to get the homologous idrs. However, you can generate those IDRs in any way you like.

In [5]:
import pairk.backend.tools.sequence_utils as tools
import pairk.examples
faimporter = tools.FastaImporter(pairk.examples.example1_alignment_file)
alignment = faimporter.import_as_alignment()
idrs = tools.strip_dashes_from_sequences(list(alignment[:, 555:971]))
idr_dict = {i.id: str(i.seq) for i in idrs}
query_id = "9606_0:00294e"

# step 1: Pairk alignment

## scoring matrix

These methods use a scoring matrix to score the query k-mer to homolog k-mer matches.

### method - exhaustive scoring - slow and not recommended for general use

The exhaustive scoring method actually scores all possible query k-mer to ortho k-mer matches. It's a brute force method that is generally slower than the needleman variant (see the next method). The results should actually be the same as the needleman method. I've included it in the `pairk` package in case people want to use the exhaustive method for custom analysis, such as including multiple high-scoring matches from each ortholog, etc.

all you need to run the method is:
- the idr sequences in dictionary format (sequence id as key, sequence string as value)
- the sequence id of the query sequence
- the k-mer length to use for the alignment (k)
- the scoring matrix to use for the alignment (default in EDSSMat50)

In [24]:
aln_results = pairk.pairk_alignment(
    idr_dict_in=ex1.idr_dict, 
    query_id=ex1.query_id,
    k=5, 
    matrix_name="EDSSMat50"
)

The results are returned as a `PairkAln` object.

In [35]:
type(aln_results)

pairk.backend.tools.pairwise_tools.PairkAln

In [37]:
pairk.PairkAln?

Init signature:
pairk.PairkAln(
    orthokmer_df: pandas.core.frame.DataFrame,
    pos_df: pandas.core.frame.DataFrame,
    score_df: pandas.core.frame.DataFrame | None = None,
    rbm_df: pandas.core.frame.DataFrame | None = None,
)
Docstring:     
A class to store the results of the pairwise alignment.

The primary data is stored in pandas dataframes. All dataframes have the
same structure. One column is the query k-mer sequence
('query_kmer'). The other columns are named as the ortholog sequence
ids. The dataframe indexes are the query k-mer start position in the
query sequence.

Attributes
----------
orthokmer_matrix : pd.DataFrame
    the best scoring k-mer from each ortholog for each query k-mer.
position_matrix : pd.DataFrame
    the start position of the best scoring k-mer from each ortholog for
    each query k-mer.
score_matrix : pd.DataFrame | None
    the alignment scores for each k-mer in the query sequence against
    the corresponding best matching ortholog k-mer.
rbm_ma

The results of the pairwise alignments are stored in pandas DataFrames, which can be directly accessed via the `PairkAln` object.

In [44]:
aln_results.orthokmer_matrix.head()

,query_kmer,9793_0:005123,1706337_0:000fc7,51337_0:001b5a,9568_0:004ae1,43346_0:004190,885580_0:00488c,10181_0:00305d,1415580_0:000900,61221_0:00105a,...,30732_0:0046dd,241271_0:0048e4,8103_0:0045e4,56723_0:00152f,210632_0:004c0c,31033_0:00264e,63155_0:004c86,7994_0:004d71,109280_0:00369f,150288_0:004e5a
0,TNLGT,TNVGT,TNLGT,TNLGT,TNLGT,TNLGT,TNLGT,TNLGT,TSLHS,VAVGT,...,TNRST,RNLPT,TNRST,TNRST,ANRST,TGAGA,TNRST,SNTST,ANFGS,DNLNS
1,NLGTV,NVGTG,NLGTV,NLGTV,NLGTV,NLGTV,NLGTV,NLGTV,NLSKV,SIVAV,...,SFGGV,NLPTN,DFGHV,DFGNV,NLPPP,ASGSV,SFGGV,NTSTP,QLPTV,NLNSQ
2,LGTVN,VGTGN,LGTVN,LGTVN,LGTVN,LGTVN,LGTVN,LGTVS,LSKVD,IVAVG,...,FGGVD,TGSVK,PGQAN,FGNVP,FGGMD,MKAMN,FGGVG,MLAMN,LPTVH,SDNLN
3,GTVNA,GTGNA,GTVNA,GTVNT,GTVNA,GTVNA,GTVNT,GTVST,GAVQA,GSTHA,...,PTVKA,GTIPP,GQANG,PTIKA,PTIKA,TTIKA,PTVKA,STSNT,PTVHS,DNLNS
4,TVNAA,TGNAA,TVNAA,TVNTA,TVNAA,TVNAV,TVNTA,TVSTA,NVNAN,AVSAG,...,TVKAK,PVNSF,PLNAL,TAAAA,TIKAK,TIKAS,TVKAK,TSNTS,TTAAA,NLNSQ


In [45]:
aln_results.orthokmer_matrix.loc[1]

query_kmer          NLGTV
9793_0:005123       NVGTG
1706337_0:000fc7    NLGTV
51337_0:001b5a      NLGTV
9568_0:004ae1       NLGTV
43346_0:004190      NLGTV
885580_0:00488c     NLGTV
10181_0:00305d      NLGTV
1415580_0:000900    NLSKV
61221_0:00105a      SIVAV
7897_0:0033c5       NFAKV
8407_0:002bff       NVANV
173247_0:004550     HFGGM
30732_0:0046dd      SFGGV
241271_0:0048e4     NLPTN
8103_0:0045e4       DFGHV
56723_0:00152f      DFGNV
210632_0:004c0c     NLPPP
31033_0:00264e      ASGSV
63155_0:004c86      SFGGV
7994_0:004d71       NTSTP
109280_0:00369f     QLPTV
150288_0:004e5a     NLNSQ
Name: 1, dtype: object

In [46]:
aln_results.score_matrix.head()

,query_kmer,9793_0:005123,1706337_0:000fc7,51337_0:001b5a,9568_0:004ae1,43346_0:004190,885580_0:00488c,10181_0:00305d,1415580_0:000900,61221_0:00105a,...,30732_0:0046dd,241271_0:0048e4,8103_0:0045e4,56723_0:00152f,210632_0:004c0c,31033_0:00264e,63155_0:004c86,7994_0:004d71,109280_0:00369f,150288_0:004e5a
0,TNLGT,22.0,28.0,28.0,28.0,28.0,28.0,28.0,10.0,9.0,...,13.0,12.0,13.0,13.0,9.0,8.0,13.0,8.0,14.0,9.0
1,NLGTV,16.0,29.0,29.0,29.0,29.0,29.0,29.0,16.0,8.0,...,11.0,13.0,11.0,13.0,7.0,7.0,11.0,6.0,14.0,10.0
2,LGTVN,16.0,29.0,29.0,29.0,29.0,29.0,23.0,10.0,7.0,...,11.0,8.0,9.0,10.0,7.0,8.0,10.0,8.0,16.0,4.0
3,GTVNA,19.0,26.0,23.0,26.0,26.0,23.0,17.0,15.0,10.0,...,11.0,9.0,10.0,8.0,8.0,9.0,11.0,9.0,9.0,6.0
4,TVNAA,18.0,25.0,22.0,25.0,22.0,22.0,16.0,14.0,11.0,...,11.0,9.0,8.0,12.0,8.0,11.0,11.0,10.0,11.0,5.0


You can get the "pseudo-alignment" of any query k-mer via the `get_pseudo_alignment` method. <br>This method returns a list of the best-scoring ortholog k-mers for a query k-mer. The query k-mer is specified by its position in the query sequence (0-based). 
<br>The returned list includes the query k-mer sequence

In [40]:
aln_results.get_pseudo_alignment(1)

['NLGTV',
 'NVGTG',
 'NLGTV',
 'NLGTV',
 'NLGTV',
 'NLGTV',
 'NLGTV',
 'NLGTV',
 'NLSKV',
 'SIVAV',
 'NFAKV',
 'NVANV',
 'HFGGM',
 'SFGGV',
 'NLPTN',
 'DFGHV',
 'DFGNV',
 'NLPPP',
 'ASGSV',
 'SFGGV',
 'NTSTP',
 'QLPTV',
 'NLNSQ']

you can search for a specific kmer to get its positions. You can then use the positions to query the matrices.

In [41]:
aln_results.find_query_kmer_positions('LPPPP')

[75, 113, 127, 157]

In [29]:
aln_results.get_pseudo_alignment(75)

['LPPPP',
 'LPPPP',
 'LPPPP',
 'LPPPP',
 'PPMPP',
 'LPPPP',
 'LPDRP',
 'APSPP',
 'LPPPP',
 'LPPPP',
 'LPPPP',
 'LPPPP',
 'LPPPP',
 'LPPPP',
 'LPPPP',
 'LPPPP',
 'LPPPP',
 'LPPPP',
 'LPPPP',
 'LPPPP',
 'LPPPP',
 'LPPPP',
 'IPPPP']

In [48]:
aln_results.orthokmer_matrix.loc[[75, 113, 127, 157]].T

,75,113,127,157
query_kmer,LPPPP,LPPPP,LPPPP,LPPPP
9793_0:005123,LPPPP,LPPPP,LPPPP,LPPPP
1706337_0:000fc7,LPPPP,LPPPP,LPPPP,LPPPP
51337_0:001b5a,LPPPP,LPPPP,LPPPP,LPPPP
9568_0:004ae1,PPMPP,PPMPP,PPMPP,PPMPP
43346_0:004190,LPPPP,LPPPP,LPPPP,LPPPP
885580_0:00488c,LPDRP,LPDRP,LPDRP,LPDRP
10181_0:00305d,APSPP,APSPP,APSPP,APSPP
1415580_0:000900,LPPPP,LPPPP,LPPPP,LPPPP
61221_0:00105a,LPPPP,LPPPP,LPPPP,LPPPP


Note - the k-mers are defined by position rather than sequence. You could easily make a variant of this method that uses the unique sequences instead. It would make the method slightly faster. <br>The reason that I didn't do this is because I wanted to mimic the LLM embedding version of Pairk, where identical k-mers have different embeddings and thus are treated as different k-mers.<br>Inclusion of duplicate k-mers does alter the final z-scores, so it's something to be aware of.

In [6]:
aligner = pairk.make_aligner('EDSSMat50')
r = pairk.pairk_alignment_needleman(idr_dict, query_id, 10, aligner=aligner)

In [8]:
import pairk.single_kmer as pairk_single
pairk_single.pairk_alignment_single_kmer("LPPPP", idr_dict)

ValueError: Multiple best scores found